In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as ort

In [2]:
# Generate random data
X, y = make_regression(n_samples=100, n_features=4, random_state=42)

# Train a Random Forest model
rf = RandomForestRegressor(n_estimators=10, random_state=42)
rf.fit(X, y)

# Convert to ONNX
initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = convert_sklearn(rf, initial_types=initial_type, target_opset=13)

# Save the model
onnx.save_model(onnx_model, "rf_model.onnx")

# Verify the model

# Create test input
test_input = np.array([[1.2, 0.5, 3.4, 2.0], [2.3, 1.1, 4.1, 1.5]], dtype=np.float32)

# Run inference with ONNX Runtime
session = ort.InferenceSession("rf_model.onnx")
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

onnx_prediction = session.run([output_name], {input_name: test_input})[0]
sklearn_prediction = rf.predict(test_input)

print("ONNX prediction:", onnx_prediction)
print("Scikit-learn prediction:", sklearn_prediction)

ONNX prediction: [[ 91.10057]
 [181.9491 ]]
Scikit-learn prediction: [ 91.10057206 181.9491077 ]
